In [7]:
# -*- coding: utf-8 -*-
string = u"我 爱 你        "
print(string.split(' '))

[u'\u6211', u'\u7231', u'\u4f60', u'', u'', u'', u'', u'', u'', u'', u'']


In [9]:
import os
import jieba
import random
import numpy
def read_document(doc_num,stop_words,dic):
    doc = []
    for m in range(doc_num):
        doc.append([])
        path = os.getcwd() + "\\data\\C000007\\1"+str(m)+".txt"
        #path = os.getcwd() + "\\data\\1"+str(m)+".txt"
        f = open(path)
        try:
            ftext = f.read()

        finally:
            f.close()                  
        f_seg_list = jieba.cut(ftext)  
        #liststr="/".join(f_seg_list)
        
        term_id = 0
        for myword in f_seg_list:
        
            if not(myword.strip() in stop_words) and len(myword.strip())>1:
                dic.setdefault(myword,0)
               
                doc[m].append(myword)
        for key in dic.keys():
            dic[key] = term_id
            term_id += 1
    return doc,dic
        
        

            
  
    
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list



def gibbs_sampling(z, m, i, nt, nd, nt_sum, nd_sum, term):
    topic = z[m][i]      #当前主题
    nt[term][topic] -= 1 #去除当前词
    nd[m][topic] -= 1    
    nt_sum[topic] -= 1
    nd_sum[m] -= 1

    topic_alpha = topic_number * alpha
    term_beta = len(dic) * beta
    p = [0 for x in range(topic_number)]   #p[k]: 属于主题k的概率
    for k in range(topic_number):
        p[k] = (nd[m][k] + alpha) / (nd_sum[m] + topic_alpha) \
                * (nt[term][k] + beta) / (nt_sum[k] +term_beta)
        if k >= 1:             #转换成累加概率
            p[k] += p[k-1]
    gs = random.random() * p[topic_number-1]  #随机采样
    new_topic = 0
    while new_topic < topic_number:
        if p[new_topic] > gs:
            break
        new_topic += 1
    nt[term][new_topic] += 1
    nd[m][new_topic] += 1
    nt_sum[new_topic] += 1
    nd_sum[m] += 1
    z[m][i] = new_topic #新主题   
    
def calc_theta(nd, nd_sum): #每个文档的主题分布
    doc_num = len(nd)
    topic_alpha = topic_number * alpha
    theta = [[0 for t in range(topic_number)] for d in range(doc_num)]
    for m in range(doc_num):
        for k in range(topic_number):
            theta[m][k] = (nd[m][k] + alpha) / (nd_sum[m] + topic_alpha)
    return theta

def calc_phi(nt, nt_sum):  #每个主题的词分布
    term_num = len(nt)
    term_beta = term_num * beta
    phi = [[0 for w in range(term_num)] for t in range(topic_number)]
    for k in range(topic_number):
        for term in range(term_num):
            phi[k][term] = (nt[term][k] + beta) / (nt_sum[k] + term_beta)
    return phi

def lda(z, nt, nd, nt_sum, dic, doc):
    doc_num = len(z)    #文档数
    for time in range(10000):
        for m in range(doc_num):
            doc_length = len(z[m])  #第m篇文档的长度
            for i in range(doc_length):
                term = dic[doc[m][i]]  #第m篇文档的第i个词 --> 词汇
                gibbs_sampling(z, m, i, nt, nd, nt_sum, nd_sum, term)
        theta = calc_theta(nd, nd_sum)   #计算每个文档的主题分布
        phi = calc_phi(nt, nt_sum)       #计算每个主题的的词分布
        return theta,phi


                
def init_topic(doc, nt, nd, nt_sum, nd_sum, dic):
    #随机分配类型
    
    #topic_number = len(nd[m]) 

    doc_num = len(nd)
    z = [[0 for x in range(len(doc[m]))] for m in range(doc_num)]
    for x in range(doc_num):
        nd_sum[x] = len(doc[x])
        for y in range(len(doc[x])):
            topic = random.randint(0,topic_number-1)
            z[x][y] = topic
            term = dic.get(doc[x][y])
            nt[term][topic] += 1
            nd[x][topic] += 1
            nt_sum[topic] += 1

    return z                

def show_result(theta, phi, dic):
    term_num = len(dic)
    doc_num = len(theta)
    topic_num = len(phi)
    dics = []
    dics = list(dic.items())
    #print(dics)
    dics.sort(key = lambda i:i[1],reverse = False)
    #print(dics)
    #排序后的词元组
    
    topic_word=[]
    for t in range(topic_num):
        topic_word.append([])
        print("主题" +str(t)+ ": ",end = " ")
        top_terms = []
        top_terms = [(n,phi[t][n]) for n in range(term_num)]
        top_terms.sort(key = lambda i:i[1],reverse = True)
        #排序后最优主题词汇
        for top in range(top_num):
            top_word = dics[top_terms[top][0]][0]
            topic_word[t].append(top_word)
            #top_word = sorted_dic({value:key for key, value in dic.items()})[top_terms[top][0]]
            print(top_word +"(" + str(top_terms[top][1]) +") ",end = "")
            
            
        print("\n")
    doc_topic = []
    for m in range(doc_num):
        print("文档"+str(m)+": ",end = "")
        
        doc_topic.append([])
        doc_topic[m] = [(k,theta[m][k]) for k in range(topic_num)]
        doc_topic[m].sort(key = lambda i:i[1],reverse = True)
       
        print(doc_topic[m][0],end = " ")
        tid = doc_topic[m][0][0]
        print(topic_word[tid],end = " ")
        
        print("\n")      
    #print(topic_word) 
    #print(doc_topic)
        

if __name__ == "__main__":
    doc_num = 3#文档数目
    #载入停用词表
    stop_words = load_stopwords()
    dic = {}
    doc,dic = read_document(doc_num,stop_words,dic)
    #print(len(doc))
    #print(len(dic))
    #LDA
    
 
    topic_number = 6 #主题数
    term_num = len(dic)        #词汇数
    top_num = 3               #主题关键词数
    alpha = 50/topic_number
    beta = 0.01
    # nt[w][t]：第term个词属于第t个主题的次数
    nt = [[0 for t in range(topic_number)] for term in range(term_num)]
    # nd[d][t]: 第d个文档中出现第t个主题的次数
    nd = [[0 for t in range(topic_number)] for d in range(doc_num)]
    # nt_sum[t]: 第t个主题出现的次数(nt矩阵的第t列)
    nt_sum = [0 for t in range(topic_number)]
    # nd_sum[d]: 第d个文档的长度（nd矩阵的第d行）
    nd_sum = [0 for d in range(doc_num)]
    
    z = init_topic(doc, nt, nd, nt_sum, nd_sum, dic)
    theta, phi = lda(z, nt, nd, nt_sum, dic, doc)
    # 输出每个文档的主题和每个主题的关键字
    show_result(theta, phi, dic)
    #print(doc[1])
    #print(z[1])
    #print(theta)
    #print(phi)
    #print(dic)
    #print(stop_words)

    #print(theta[5])
    #print('虽然' in dic)
    #print('已经' in stop_words)

主题0:  30(0.01889042299485377) 企业(0.01889042299485377) 费用(0.01889042299485377) 

主题1:  汽车(0.029240107388817552) 强制(0.029240107388817552) 资金(0.02340375860861445) 

主题2:  车主(0.029411764705882353) 发展(0.029411764705882353) 陆风(0.023541152988141363) 

主题3:  利润(0.03657052452233175) 奥克斯(0.03048557867834976) 三者(0.02440063283436777) 

主题4:  新飞(0.05046889180195623) 责任(0.03534334980336795) 投保(0.03030150247050519) 

主题5:  强制保险(0.034872925612162) 限额(0.029070442149239872) 保费(0.029070442149239872) 

文档0: (4, 0.24905087319665906) ['新飞', '责任', '投保'] 

文档1: (2, 0.19962335216572505) ['车主', '发展', '陆风'] 

文档2: (5, 0.19423558897243107) ['强制保险', '限额', '保费'] 



In [18]:
from sklearn import svm
X = theta[0:5]
Y = [0,0,0,1,1]
clf = svm.SVC()
#clf = svm.SVC(decision_function_shape ='ovo')
clf.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [19]:
X

[[0.2642369020501139,
  0.2072892938496583,
  0.13895216400911162,
  0.20956719817767655,
  0.17995444191343962],
 [0.13559322033898305,
  0.2598870056497175,
  0.14689265536723164,
  0.2711864406779661,
  0.1864406779661017],
 [0.16917293233082706,
  0.21804511278195488,
  0.19924812030075187,
  0.22180451127819548,
  0.19172932330827067],
 [0.17857142857142858,
  0.13214285714285715,
  0.2892857142857143,
  0.22142857142857142,
  0.17857142857142858],
 [0.3240418118466899,
  0.13937282229965156,
  0.1672473867595819,
  0.22996515679442509,
  0.13937282229965156]]

In [20]:
clf.predict([[0.1732283464566929, 0.1968503937007874, 0.23622047244094488, 0.18110236220472442, 0.2125984251968504]])

array([0])

In [10]:
theta

[[0.15793470007593013,
  0.1533788914198937,
  0.13515565679574792,
  0.17160212604403946,
  0.24905087319665906,
  0.1328777524677297],
 [0.1657250470809793,
  0.18832391713747648,
  0.19962335216572505,
  0.13747645951035783,
  0.14312617702448213,
  0.1657250470809793],
 [0.14912280701754385,
  0.16791979949874686,
  0.17731829573934837,
  0.15664160401002505,
  0.15476190476190474,
  0.19423558897243107]]

In [1]:
import gensim
help(gensim)

Help on package gensim:

NAME
    gensim

DESCRIPTION
    This package contains interfaces and functionality to compute pair-wise document
    similarities within a corpus of documents.

PACKAGE CONTENTS
    corpora (package)
    interfaces
    matutils
    models (package)
    nosy
    parsing (package)
    scripts (package)
    similarities (package)
    summarization (package)
    test (package)
    utils

CLASSES
    logging.Handler(logging.Filterer)
        NullHandler
    
    class NullHandler(logging.Handler)
     |  For python versions <= 2.6; same as `logging.NullHandler` in 2.7.
     |  
     |  Method resolution order:
     |      NullHandler
     |      logging.Handler
     |      logging.Filterer
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  emit(self, record)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from logging.Handler:
     |  
     |  __init__(self, level=0)


In [2]:
from gensim import models
help(models)
help(models.ldamodel)

Help on package gensim.models in gensim:

NAME
    gensim.models

DESCRIPTION
    This package contains algorithms for extracting document representations from their raw
    bag-of-word counts.

PACKAGE CONTENTS
    doc2vec
    hdpmodel
    lda_dispatcher
    lda_worker
    ldamodel
    ldamulticore
    logentropy_model
    lsi_dispatcher
    lsi_worker
    lsimodel
    phrases
    rpmodel
    tfidfmodel
    word2vec
    wrappers (package)

CLASSES
    gensim.interfaces.TransformationABC(gensim.utils.SaveLoad)
        VocabTransform
    
    class VocabTransform(gensim.interfaces.TransformationABC)
     |  Remap feature ids to new values.
     |  
     |  Given a mapping between old ids and new ids (some old ids may be missing = these
     |  features are to be discarded), this will wrap a corpus so that iterating over
     |  `VocabTransform[corpus]` returns the same vectors but with the new ids.
     |  
     |  Old features that have no counterpart in the new ids are discarded. This

In [1]:
help(models.ldamodel)

NameError: name 'models' is not defined

In [21]:
#-*- coding:utf-8 –*-
import os
import jieba
all_words={}
f_stop = open('stopwords.txt')
try:
    f_stop_text = f_stop.read()
    
finally:
    f_stop.close()

f_stop_seg_list = f_stop_text.split('\n')

#print(f_stop_seg_list)

doc = []
for i in range(3):
    doc.append([])
    path = os.getcwd() + "\\data\\1"+str(i)+".txt"
    f = open(path)
    try:
        f_text = f.read()

    finally:
        f.close()  
    #print(f_text)
    f_seg_list = jieba.cut(f_text)
    
   
    word_id = 0    
   
    
    for myword in f_seg_list:
        
        if not(myword.split() in f_stop_seg_list)and len(myword.strip())>1:
            
            all_words.setdefault(myword,0)
            doc[i].append(myword)
            #all_words[myword] += 1
    term_id = 0
    for key in all_words.keys():
        all_words[key] = term_id
        term_id += 1
    
#print(doc)
print(len(all_words))
print(all_words['政府部门'])
print(all_words[doc[0][0]])
nt = [[0 for t in range(10)] for term in range(704)]
for m in range(3):
    print(len(doc[m]))
    for n in range(len(doc[m])):
        nt.append(1)
        term = all_words.get(doc[m][n])
        #nt[term][topic] += 1
        #print(term,end = " ")
    print("\n")
#print(all_words)
#print(nt[term])


#print(all_words.values())    
sortdic = list(all_words.items())
sortdic.sort(key = lambda i:i[1],reverse = True)
print(len(sortdic))


704
617
617
422


137


553


704


In [22]:
help(gensim)

NameError: name 'gensim' is not defined

In [1]:
import numpy as np
import jieba
import copy
ftest1fn = '10.txt'
ftest2fn = '11.txt'
sampfn = '12.txt'

def get_cossimi(x,y):
    myx = np.array(x)
    myy = np.array(y)
    cos1 = np.sum(myx*myy)
    cos21 =  np.sqrt(sum(myx*myx))
    cos22 = np.sqrt(sum(myy*myy))
    return cos1/float(cos21*cos22)

#if __name__ == "__main__":
print()
print('loading...')
print('working')
f1 = open(sampfn)
try:
    f1_text = f1.read()
finally:
    f1.close()
f1_seg_list = jieba.cut(f1_text)


"""
ftest1 = open(ftest1fn)
try:
    ftest1_text = ftest1.read()
finally:
    ftest1.close()
ftest1_seg_list = jieba.cut(ftest1_text)



ftest2 = open(ftest2fn)
try:
    ftest2_text = ftest2.read()
finally:
    ftest2.close()
ftest2_seg_list = jieba.cut(ftest2_text)

"""
f_stop = open('stopwords.txt')
try:
    f_stop_text = f_stop.read()

finally:
    f_stop.close()

f_stop_seg_list = f_stop_text.split('\n')

test_words = {}
all_words = {}

for myword in f1_seg_list:
    print('.',end="")
    if not(myword.split() in f_stop_seg_list)and len(myword.strip())>1:
        test_words.setdefault(myword,0)
        all_words.setdefault(myword,0)
        all_words[myword] += 1
print(all_words)
"""
          
    mytest1_words = copy.deepcopy(test_words)
    for myword in ftest1_seg_list:
        print ('.',end="")
        if not(myword.strip() in f_stop_seg_list):
            if myword in mytest1_words.keys():
            #if mytest1_words.has_key(myword):
                mytest1_words[myword]+= 1
    mytest2_words = copy.deepcopy(test_words)
    for myword in ftest2_seg_list:
        print ('.',end ="")
      
        if not(myword.strip() in f_stop_seg_list):
            if myword in mytest1_words.keys():
            #if mytest2_words.has_key(myword):
                mytest2_words[myword]+= 1
                
    sampdata=[]
    test1data=[]
    test2data=[]
    for key in all_words.keys():
        sampdata.append(all_words[key])
        test1data.append(mytest1_words[key])
        test2data.append(mytest2_words[key])
    test1simi = get_cossimi(sampdata,test1data)
    test2simi = get_cossimi(sampdata,test2data)

    print(u"\n%s与样本[%s]的余弦相似度：%f" % (ftest1fn,sampfn,test1simi))
    print(u"%s与样本[%s]的余弦相似度：%f" % (ftest2fn,sampfn,test2simi))  
"""  

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 1.796 seconds.
DEBUG:jieba:Loading model cost 1.796 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.



loading...
working
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

'\n          \n    mytest1_words = copy.deepcopy(test_words)\n    for myword in ftest1_seg_list:\n        print (\'.\',end="")\n        if not(myword.strip() in f_stop_seg_list):\n            if myword in mytest1_words.keys():\n            #if mytest1_words.has_key(myword):\n                mytest1_words[myword]+= 1\n    mytest2_words = copy.deepcopy(test_words)\n    for myword in ftest2_seg_list:\n        print (\'.\',end ="")\n      \n        if not(myword.strip() in f_stop_seg_list):\n            if myword in mytest1_words.keys():\n            #if mytest2_words.has_key(myword):\n                mytest2_words[myword]+= 1\n                \n    sampdata=[]\n    test1data=[]\n    test2data=[]\n    for key in all_words.keys():\n        sampdata.append(all_words[key])\n        test1data.append(mytest1_words[key])\n        test2data.append(mytest2_words[key])\n    test1simi = get_cossimi(sampdata,test1data)\n    test2simi = get_cossimi(sampdata,test2data)\n\n    print(u"\n%s与样本[%s]的余弦相似度：

In [78]:
L = []
k = [1,2,1,2]
for x in range(1, 11):
    L.append(x * x)
    L.append(k)
print(L[1][1])

2


In [122]:
theta = [[0 for t in range(3)] for d in range(5)]
theta[1] = [1,2,3]
for m in range(5):
    theta[m].sort(reverse = True)
    print(theta[m])

print(theta)
help(theta.sort)

[0, 0, 0]
[3, 2, 1]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[[0, 0, 0], [3, 2, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
Help on built-in function sort:

sort(...) method of builtins.list instance
    L.sort(key=None, reverse=False) -> None -- stable sort *IN PLACE*



#实例4: 对第二个关键字排序 

L = [('b',6),('a',1),('c',3),('d',4)]

L.sort(key=lambda x:x[1]) 

L

[('a', 1), ('c', 3), ('d', 4), ('b', 6)]

In [128]:
L = [('b',6),('a',1),('c',3),('d',4)]

L.sort(key=lambda x:x[1]) 

print(L[-1][1])

6


In [1]:
import numpy
import random
#遍历文件夹：
import os
want_dir = '.'
for root, dirs, files in os.walk(want_dir):
        for file in files:
                if os.path.splitext(file)[1] == '.py':
                        filename = os.path.join(root,file)
                        print filename

In [16]:
import sklearn

from sklearn import lda
help(lda)

Help on module sklearn.lda in sklearn:

NAME
    sklearn.lda - Linear Discriminant Analysis (LDA)

CLASSES
    sklearn.base.BaseEstimator(builtins.object)
        LDA(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.base.TransformerMixin)
    sklearn.base.TransformerMixin(builtins.object)
        LDA(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.base.TransformerMixin)
    sklearn.linear_model.base.LinearClassifierMixin(sklearn.base.ClassifierMixin)
        LDA(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.base.TransformerMixin)
    
    class LDA(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.base.TransformerMixin)
     |  Linear Discriminant Analysis (LDA).
     |  
     |  A classifier with a linear decision boundary, generated by fitting class
     |  conditional densities to the data and using Bayes' rule.
     |  
     |  The model

In [14]:
dic = {}
help(dic)
dic = dict(one=1, two =2)
print(dic)

(x for x in range(10))

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if D has a key k, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize s